Problem:

1. Given the training set and the test set files from the Program 1 assignment, extract features from both using the standard tools and the mannual function (see above).

2. Make sure that both document-feature matrices need to be in the same Euclidean space, i.e., the ith dimension refers to the same token in both the training and test matrices. Think about different ways you could achieve this with the manual processing function.

3. Time processing the datasets using both the standard tools and the mannual method and report the times.

USING STANDARD TOOLS 

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from scipy.sparse import csr_matrix
import string
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer
import math
import time

In [2]:
def getDocLength(filename):
    file = open(filename,"r")
    
    len = 0
    
    for line in file:
        len += 1
        
    return len

file = open("both.dat","r")

doc = []
for line in file:
    doc.append(line)

start1 = time.time()
cv = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
count_occurrences = cv.fit_transform(doc)

dataArray = count_occurrences.toarray()
testLength = getDocLength("test.dat")

count_vect_df = pd.DataFrame(data = dataArray[:testLength], columns= cv.get_feature_names_out())

display(count_vect_df)


,0,00,000,000016447,00010563n,0002,0004287,0005173miami050,000660,0009875buffalo040,...,zwick,zwiki,zy,zydrunas,zygmunt,zyman,zyprexa,zz,zzz,zzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25515,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25516,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25517,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25518,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
count_vect_df2 = pd.DataFrame(data = dataArray[testLength:], columns = cv.get_feature_names_out())

display(count_vect_df2)

end1 = time.time()
print(f"Time to process using standard tools: {end1 - start1} seconds")

,0,00,000,000016447,00010563n,0002,0004287,0005173miami050,000660,0009875buffalo040,...,zwick,zwiki,zy,zydrunas,zygmunt,zyman,zyprexa,zz,zzz,zzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102075,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102076,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102077,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102078,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Time to process using standard tools: 13.381695032119751 seconds


USING MANUAL FUNCTION

In [4]:
import re

def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    #nnz = number of non zero values 
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
    
    # set up memory
    ind = np.zeros(nnz, dtype=int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
        
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat,idx

def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat
        
def textToMatrix(docs):
    return build_matrix(docs)

def getDoc(filename):
    file = open(filename,"r")
    
    doc = []
    for line in file:
        lowerline = line.lower()
        nospecialchar = re.sub(r'[^a-zA-Z0-9_]', ' ', lowerline)
        words = nospecialchar.split()
        for w in words:
            if len(w) == 1:
                words.remove(w)
        doc.append(words)
    
    return doc  

start2 = time.time()
mat,manualidx = textToMatrix(getDoc("both.dat"))
end2 = time.time()

testmat = mat[:testLength]
trainmat = mat[testLength:]

print("Test Matrix:")
print(csr_info(testmat))

print("\nTrain Matrix:")
print(csr_info(trainmat))

print(f"\nTime to process using manual function: {end2 - start2} seconds")

Test Matrix:
 [nrows 25520, ncols 66494, nnz 798990]
None

Train Matrix:
 [nrows 102080, ncols 66494, nnz 3196193]
None

Time to process using manual function: 7.186454772949219 seconds
